# Import Library

In [ ]:
!pip install -q stockstats

In [ ]:
!pip install -q catboost

     |████████████████████████████████| 76.6 MB 45 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"
model_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Catboost/"

Mounted at /content/drive


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import floor
import pickle
from catboost import CatBoostRegressor

import torch.nn as nn
import os.path as osp
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
import time, matplotlib, torch, os
from statsmodels.tsa.arima_model import ARIMA
from torch.utils.tensorboard import SummaryWriter

In [21]:
import os
from os import walk

import plotly
%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime


In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [23]:
#For DNN / CNN+LSTM
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Flatten, Dense

# Import df

In [24]:
import pandas_datareader.data as web
from stockstats import StockDataFrame

#Setting the index values as timestamp
def process(code):
  print("Enter the code of the model that you want to implement")
  #code = input()
  #final_code = '^'+code
  final_code = code
  start_date = "2012-01-01"
  end_date = "2021-12-31"
  stock = web.DataReader(final_code , 'stooq',start=start_date, end=end_date)
  stock.columns = ["open","high","low","close","volume"]
  
  # Use online package to generate additional features
  x = StockDataFrame(stock)
  stock = x[['open','high','low','close','volume',
          'boll', 'boll_ub', 'boll_lb',
          'macd', 'macdh', 'macds',
          'rsi_11', 'rsi_14', 'rsi_21']]
  stock.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in stock.index]
  stock = stock.sort_index()

  return stock

In [25]:
stock = process("IBM")

Enter the code of the model that you want to implement


In [26]:
stock

,open,high,low,close,volume,boll,boll_ub,boll_lb,macd,macdh,macds,rsi_11,rsi_14,rsi_21
20120103,128.88,130.23,128.38,128.58,8180058,128.1790,134.932612,121.425388,-1.811274,0.489102,-2.300376,51.281366,47.190995,42.920670
20120104,128.10,128.60,127.63,128.07,6297526,128.3965,135.476771,121.316229,-2.091832,0.330819,-2.422652,48.488535,44.808561,41.192557
20120105,127.56,127.71,126.38,127.46,6466200,128.6025,135.879775,121.325225,-2.367574,0.137783,-2.505356,45.064237,41.897279,39.091982
20120106,127.28,127.34,125.83,125.98,7094969,128.9115,136.502568,121.320432,-2.622973,-0.083171,-2.539802,35.625325,34.061158,33.612565
20120109,125.76,125.80,124.42,125.33,7535572,129.2665,136.943913,121.589087,-2.760113,-0.241104,-2.519009,30.883945,30.223364,31.016798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,129.08,130.08,128.40,130.05,4346229,131.4500,133.282321,129.617679,-0.099397,-0.058552,-0.040846,8.180361,8.545592,8.997984
20211228,130.03,131.63,130.02,131.05,3488601,131.8000,132.899939,130.700061,-0.042801,-0.026749,-0.016052,14.491914,14.911940,15.424720
20211229,130.82,132.61,130.73,131.76,4291007,132.0500,132.611070,131.488930,-0.010219,-0.007870,-0.002349,28.868360,29.305681,29.832936
20211230,132.16,132.77,131.74,132.32,3196152,132.1950,132.548553,131.841447,0.005609,0.002493,0.003116,100.000000,100.000000,100.000000


# Processing Functions


## Split the train and test data

In [27]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

In [28]:
train_X = custom_split(stock,start = 20130101,end = 20171031)
valid_X = custom_split(stock,start = 20171101,end = 20181231)
test_X = custom_split(stock,start = 20190101,end = 20201231)

In [29]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))

1218
292
505


## Label the target result (opening price on 6th day)

In [30]:
# Assume we use 5 days price data to predict closing price of the 6th day
num_day_to_predict = 5

In [31]:
def produce_result_target_price(X,num_day = 5,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(5,len(X)):
        y.loc[X.index[i-1],result_col_name] = X.loc[X.index[i],"open"]
    
    return y

In [32]:
train_y = produce_result_target_price(train_X,num_day_to_predict)
valid_y = produce_result_target_price(valid_X,num_day_to_predict)
test_y = produce_result_target_price(test_X,num_day_to_predict)

In [33]:
print(len(train_y))
print(len(valid_y))
print(len(test_y))

1218
292
505


In [37]:
pred_y = produce_result_target_price(stock,num_day_to_predict)

## Transform the 5-day data into one vector

In [38]:
def transform_x_data_to_one_vector(X,num_day):
    col_name = []
    for i in range(num_day):
        for j in X.columns:
            col_name.append(j+"-"+str(i))
    new_X = pd.DataFrame(np.nan, index=X.index, columns=col_name)
    
    for i in range(num_day-1,len(X)):
        for col in col_name:
            split_list = col.split("-")
            new_X.loc[X.index[i],col] = X.loc[X.index[i-int(split_list[1])],split_list[0]]
    
    return new_X

In [36]:
train_X_5 = transform_x_data_to_one_vector(train_X,num_day_to_predict)
valid_X_5 = transform_x_data_to_one_vector(valid_X,num_day_to_predict)
test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)

In [39]:
pred_X_5 = transform_x_data_to_one_vector(stock,num_day_to_predict)

In [40]:
print(len(train_X_5))
print(len(valid_X_5))
print(len(test_X_5))
print(len(pred_X_5))

1218
292
505
2517


## Drop out rows with NaN

In [41]:
def drop_nan_row_y(y,num_day = 5):
    drop_list = [y.index[i] for i in range(num_day-1)]
    drop_list.append(y.index[-1])
    return y.drop(drop_list)

def drop_nan_row_X_5(X,num_day = 5):
    drop_list = [X.index[i] for i in range(num_day-1)]
    drop_list.append(X.index[-1])
    return X.drop(drop_list)

In [42]:
new_train_X_5 = drop_nan_row_X_5(train_X_5,num_day_to_predict)
new_train_y = drop_nan_row_y(train_y,num_day_to_predict)

new_valid_X_5  = drop_nan_row_X_5(valid_X_5,num_day_to_predict)
new_valid_y = drop_nan_row_y(valid_y,num_day_to_predict)

new_test_X_5  = drop_nan_row_X_5(test_X_5,num_day_to_predict)
new_test_y = drop_nan_row_y(test_y,num_day_to_predict)

In [43]:
new_pred_X_5  = drop_nan_row_X_5(pred_X_5,num_day_to_predict)
new_pred_y = drop_nan_row_y(pred_y,num_day_to_predict)

In [44]:
print(len(new_train_X_5))
print(len(new_valid_X_5))
print(len(new_test_X_5))
print(len(new_pred_X_5))

1213
287
500
2512


In [45]:
print(len(new_train_y))
print(len(new_valid_y))
print(len(new_test_y))
print(len(new_pred_y))

1213
287
500
2512


In [46]:
# Keep a copy for reverting prediction result
old_train_X_5 = new_train_X_5
old_train_y = new_train_y
old_valid_X_5 = new_valid_X_5
old_valid_y = new_valid_y
old_test_X_5 = new_test_X_5
old_test_y = new_test_y

In [47]:
old_pred_X_5 = new_pred_X_5
old_pred_y = new_pred_y

## Normalize data row by row

In [48]:
def normalize_data_by_row(X,y):
    norm_X = X.sub(X.mean(axis=0), axis=1).div(X.std(axis=0), axis=1)
    
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    
    temp_y = y.loc[:,"result_price"]
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = temp_y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = temp_y.index
    

    norm_y = pd.DataFrame((temp_y-mean_5_day)/std_5_day,
                          columns=["result_price"])
                         
    return (norm_X,norm_y)

In [49]:
new_train_X_5,new_train_y = normalize_data_by_row(new_train_X_5,new_train_y)
new_valid_X_5,new_valid_y = normalize_data_by_row(new_valid_X_5,new_valid_y)
new_test_X_5,new_test_y = normalize_data_by_row(new_test_X_5,new_test_y)

In [50]:
new_pred_X_5,new_pred_y = normalize_data_by_row(new_pred_X_5,new_pred_y)

In [51]:
new_train_X_5 = drop_nan_row_X_5(new_train_X_5,num_day_to_predict)
new_valid_X_5 = drop_nan_row_X_5(new_valid_X_5,num_day_to_predict)
new_test_X_5 = drop_nan_row_X_5(new_test_X_5,num_day_to_predict)

In [52]:
new_train_y = drop_nan_row_y(new_train_y,num_day_to_predict)
new_valid_y = drop_nan_row_y(new_valid_y,num_day_to_predict)
new_test_y = drop_nan_row_y(new_test_y,num_day_to_predict)

In [53]:
new_train_X_5

,open-0,high-0,low-0,close-0,volume-0,boll-0,boll_ub-0,boll_lb-0,macd-0,macdh-0,...,volume-4,boll-4,boll_ub-4,boll_lb-4,macd-4,macdh-4,macds-4,rsi_11-4,rsi_14-4,rsi_21-4
20130114,0.801829,0.760783,0.804657,0.785525,0.061790,1.279803,1.427751,1.082076,-1.406745,-1.580071,...,-0.466321,1.174692,1.412356,0.891349,-1.474971,-0.776921,-1.319412,-1.322751,-1.378452,-1.373634
20130115,0.715955,0.727228,0.725498,0.780621,0.061379,1.304304,1.408701,1.149016,-1.298581,-1.627599,...,-0.381260,1.201114,1.421966,0.933513,-1.481067,-0.993095,-1.254608,-1.526228,-1.545811,-1.492648
20130116,0.756030,0.755054,0.781807,0.784707,-0.497657,1.329183,1.379580,1.226733,-1.138552,-1.519737,...,-0.198093,1.225358,1.417582,0.985378,-1.442880,-1.119914,-1.171975,-1.444668,-1.471530,-1.429014
20130117,0.859896,0.829532,0.890343,0.846824,-0.067296,1.352552,1.337385,1.314522,-0.921992,-1.202592,...,-0.072015,1.253790,1.420248,1.038409,-1.399309,-1.261002,-1.078883,-1.187923,-1.241027,-1.235013
20130118,0.871346,0.856540,0.921354,0.895047,0.240665,1.376592,1.294401,1.404411,-0.683810,-0.736428,...,0.063148,1.272926,1.418303,1.077832,-1.399416,-1.578803,-0.974154,-1.882927,-1.811044,-1.639197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20171023,0.441978,0.406401,0.342764,0.276328,0.453053,-0.186880,-0.234448,-0.132133,0.209160,1.457410,...,0.625585,-0.092756,0.049926,-0.231519,0.271695,0.170333,0.234060,-1.314137,-1.281820,-1.240553
20171024,0.288224,0.218979,0.053878,0.031946,1.415975,-0.212473,-0.344583,-0.072467,-0.046142,0.997030,...,10.375008,-0.079691,0.023243,-0.179297,0.769678,1.823307,0.220739,1.140162,1.083683,0.836491
20171025,0.043689,-0.001180,-0.085669,-0.123346,0.910779,-0.220528,-0.377332,-0.055597,-0.172709,0.836743,...,2.114572,-0.116833,-0.017453,-0.211431,0.693733,2.031001,0.071088,1.676509,1.547971,1.178749
20171026,-0.068355,-0.084660,-0.053026,-0.117625,-0.141216,-0.221283,-0.379406,-0.055008,-0.205863,0.937651,...,1.293473,-0.151671,-0.102481,-0.194808,0.513298,1.952254,-0.095693,2.190497,1.985631,1.492206


## Postprocessing after prediction

In [54]:
def convert_prediction(X,y,pred):
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
        
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = y.index
    a = pd.DataFrame(index = y.index,columns=['mean','std','pred'])
    a.loc[:,"mean"] = mean_5_day
    a.loc[:,"std"] = std_5_day
    a.loc[:,"pred"] = pred
    a.loc[:,"converted"] = pred*std_5_day+mean_5_day
    return a["converted"]

In [55]:
def add_5_sma(stock_):
    stock_2 = stock_
    stock_2["5_sma"] = stock_2['open'].rolling(window=5,min_periods=1).mean()
    return(stock_2)

In [56]:
def convert_decision(stock_,pred):
    h = np.array(pred)
    action = []
    status = "N"
    five_sma = stock_['open'].rolling(window=5,min_periods=1).mean()

    ind_sma = five_sma.index
    ind_pred = pred.index
    union_ind = list(ind_sma.intersection(ind_pred))

    five_sma = five_sma.filter(items = union_ind)
    pred = pred.filter(items = union_ind)

    for i in range(len(union_ind)):
      
        if pred.iloc[i] >= five_sma.iloc[i]:
            if status == "N":
                action.append("Buy")
                status = "Buy"
            else:
                action.append("Hold")
        else:
            if status == "Buy":
                action.append("Sell")
                status = "N"
            else:
                action.append("Hold")

    result = (pd.DataFrame(action,
                         columns=["Action"], index = pred.index))
                
    return result

In [57]:
def standard_date_format(date_in_num):
    date = str(date_in_num)
    new_date = date[0:4] + "-" + date[4:6] + "-" + date[6:8] 
    return new_date

# Catboost Training

In [58]:
pip install -q catboost

In [59]:
from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(random_state=1002,logging_level='Silent')

model_cat.fit(new_train_X_5, new_train_y, plot=True)
pred_cat = model_cat.predict(new_test_X_5, verbose=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [60]:
#Performance evaluation
print('CATBoost PERFORMANCE')
print('r2 score: '+str(r2_score(new_test_y, pred_cat)))
print('RMSE : '+str(np.sqrt(mean_squared_error(new_test_y, pred_cat))))
print("Mean Absolute Error : " + str(mean_absolute_error(new_test_y,pred_cat)))

CATBoost PERFORMANCE
r2 score: 0.5254815306534676
RMSE : 1.5517341798515667
Mean Absolute Error : 1.0839976510013503


In [61]:
import pickle

In [62]:
with open(model_folder+"Cat.pickle", "wb") as f:
    pickle.dump(model_cat, f)

In [63]:
cat_final_pred = model_cat.predict(new_pred_X_5)

In [64]:
cat_final_pred_converted = convert_prediction(old_pred_X_5,old_pred_y,cat_final_pred)
cat_final_pred_converted

20120109    124.970845
20120110    125.863163
20120111    125.842663
20120112    125.722518
20120113    124.862529
               ...    
20211223    131.639912
20211227    133.587578
20211228    133.281116
20211229    133.992156
20211230    135.724491
Name: converted, Length: 2512, dtype: float64

In [65]:
action_cat = convert_decision(stock,cat_final_pred_converted)
action_cat['pred_price'] = cat_final_pred_converted.tolist()

In [66]:
code = "AAPL"
file_name = f"CAT_{code}"
action_cat.to_csv(f"{file_name}.csv")

# Load Model